In [1]:
# !pip install selenium
# !pip install django
# !pip install django-embed-video

In [1]:
import csv
import uuid
from selenium import webdriver
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd

import requests
import urllib
import os

import numpy as np

In [3]:
# ----------------- Crawl data -----------------
titles  = []
years   = []
genres  = []
plots   = []
posters = []
ratings = []
trailors= []

data=[]
link_films = []
# ---------------------------------------------------

In [ ]:
s=Service('C:/Users/Van Thong/Downloads/chromedriver.exe')
driver = webdriver.Chrome(service=s) 
for i in range(1, 501):
    driver.get('https://flixgo.up.railway.app/movies-catalog/')
    your_input = driver.find_element(By.XPATH,'//div//ul/form[@method="post"]//input[@id="page"]')
    your_input.clear()
    your_input.send_keys(str(i))
    your_input.send_keys(Keys.ENTER)
    link_film=driver.find_elements(By.XPATH,"//div//a[@class='card__play']")
    for link in link_film:
        link = link.get_attribute("href")
        link_films.append({"link":link})
driver.quit()
print(link_films)

In [ ]:
for i in link_films:
    driver = webdriver.Chrome(service=s)
    driver.get(i["link"])
    
    driver.execute_script("document.querySelectorAll('body > section.section.details > div:nth-child(3) > div.row > div:nth-child(2) > div > div > div.col-12.col-sm-8.col-md-8.col-lg-9.col-xl-7 > div > ul > li:nth-child(1) > span').forEach(el => el.remove());");

    driver.execute_script("document.querySelectorAll('body > section.section.details > div:nth-child(3) > div.row > div:nth-child(2) > div > div > div.col-12.col-sm-8.col-md-8.col-lg-9.col-xl-7 > div > ul > li:nth-child(2) > span').forEach(el => el.remove());");

    # Lấy tên phim
    title = driver.find_element(By.XPATH,"//h1[@class='details__title']").text
   
    # ------------------Lấy năm phát hành------------------
    try:
        year = driver.find_element(By.XPATH,"/html/body/section[1]/div[3]/div[1]/div[2]/div/div/div[2]/div/ul/li[2]").text
        # Chỉ lấy mỗi năm
        year = year.split("-")[0]
    except:
        year = ''
    years.append({'year': year})
    # ------------------------------------------------------

    # ---------------------Lấy thể loại phim---------------------
    genre = driver.find_elements(By.XPATH,"/html/body/section[1]/div[3]/div[1]/div[2]/div/div/div[2]/div/ul/li/a")
    temp_genres={'genre':[]}

    for i in genre:
        temp_genres['genre'].append(i.text)
    genres_str = ', '.join(temp_genres['genre'])
    new_dict = {'genre': genres_str}
    genres.append(new_dict)
    # --------------------------------------------------------------

    # Lấy Overview
    try:
        plot = driver.find_element(By.XPATH,"//div[@class='card__description card__description--details']").text
    except:
        plot = ''
    # Lấy đánh giá
    try:
        rating = driver.find_element(By.XPATH,"//span[@class='card__rate']").text
    except:
        rating = ''
    # Lấy link poster 
    driver.execute_script("window.scrollTo(0, 990000)")
    poster = driver.find_element(By.XPATH,"//div[@class='card__cover']//img").get_attribute("src")
    
    # Lấy link trailer link youtobe
    # Nếu trailer không có thì sẽ trả về rổng
    try:
        trailor = driver.find_element(By.XPATH,"//div[@class='col-12 col-xl-6']//iframe").get_attribute("src")
    except:
        trailor = None

    # thêm vào list các dict tương ứng
    titles.append({'title': title})
    ratings.append({'rating': rating})
    posters.append({'poster': poster})
    plots.append({'plot': plot})
    trailors.append({'trailor': trailor})

    driver.close()

In [ ]:
# Lưu dữ liệu trên vào file csv
with open('data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['title', 'year', 'genre', 'plot', 'poster', 'rating', 'trailor'])
    for i in range(len(titles)):
        writer.writerow([titles[i]['title'], years[i]['year'], genres[i]['genre'], plots[i]['plot'], posters[i]['poster'], ratings[i]['rating'], trailors[i]['trailor']])


In [1]:
import pandas as pd

In [5]:
# Đọc file csv
df = pd.read_csv('film_DONE.csv', encoding='utf-8')
df.head()

,title,year,genre,plot,language,poster,rating,trailer,runtime
0,The Super Mario Bros. Movie,04/05/2023,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...",English,https://image.tmdb.org/t/p/w500/qNBAXBIQlnOThr...,7,https://www.youtube.com/watch?v=LTFGH0rJ-EY,103
1,Shazam! Fury of the Gods,3/15/2023,"Action, Comedy, Fantasy","Billy Batson and his foster siblings, who tran...",English,https://image.tmdb.org/t/p/w500/3GrRgt6CiLIUXU...,6,https://www.youtube.com/watch?v=gardMnJszqc,92
2,Avatar: The Way of Water,12/14/2022,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,English,https://image.tmdb.org/t/p/w500/t6HIqrRAclMCA6...,7,https://www.youtube.com/watch?v=Sw9hdSjzZjc,130
3,Creed III,03/01/2023,"Drama, Action","After dominating the boxing world, Adonis Cree...",English,https://image.tmdb.org/t/p/w500/cvsXj3I9Q2iyyI...,7,https://www.youtube.com/watch?v=qphf_-PPaaI,116
4,65,03/02/2023,"Science Fiction, Adventure, Thriller, Action","65 million years ago, the only 2 survivors of ...",English,https://image.tmdb.org/t/p/w500/rzRb63TldOKdKy...,6,https://www.youtube.com/watch?v=2kgJuEX4yhc,91


In [8]:
def convert_date(date_str):
    if isinstance(date_str, str):
        if '/' in date_str:
            day, month, year = date_str.split('/')
            return f"{month}/{day}/{year}"
        else:
            return ''
    else:
        return ''

# Điền giá trị mặc định cho các giá trị NaN hoặc Null
df['year'] = df['year'].fillna('')

# Áp dụng hàm chuyển đổi cho từng giá trị trong cột 'year'
df['year'] = df['year'].apply(convert_date)

In [11]:
# Lưu file csv
df.to_csv('film_DONE1.csv', index=False)

In [7]:
# duyệt trên cột genre và lấy các giá trị chỉ 1 lần không lặp lại
    

genre
Drama                                                      546
Comedy                                                     393
Drama, Romance                                             259
Horror                                                     248
Horror, Thriller                                           196
                                                          ... 
Animation, Comedy, Music, Family                             1
Animation, Fantasy, Family, Adventure, Comedy, TV Movie      1
Action, Drama, Comedy                                        1
Action, Crime, Science Fiction, Thriller                     1
Fantasy, Science Fiction, Family                             1
Name: count, Length: 2329, dtype: int64

In [6]:
df.head()

,title,year,genre,plot,language,poster,rating,trailer
0,The Super Mario Bros. Movie,2023-04-05,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...",English,https://image.tmdb.org/t/p/w500/qNBAXBIQlnOThr...,7,https://www.youtube.com/watch?v=LTFGH0rJ-EY
1,Shazam! Fury of the Gods,2023-03-15,"Action, Comedy, Fantasy","Billy Batson and his foster siblings, who tran...",English,https://image.tmdb.org/t/p/w500/3GrRgt6CiLIUXU...,6,https://www.youtube.com/watch?v=gardMnJszqc
2,Avatar: The Way of Water,2022-12-14,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,English,https://image.tmdb.org/t/p/w500/t6HIqrRAclMCA6...,7,https://www.youtube.com/watch?v=Sw9hdSjzZjc
3,Creed III,2023-03-01,"Drama, Action","After dominating the boxing world, Adonis Cree...",English,https://image.tmdb.org/t/p/w500/cvsXj3I9Q2iyyI...,7,https://www.youtube.com/watch?v=qphf_-PPaaI
4,65,2023-03-02,"Science Fiction, Adventure, Thriller, Action","65 million years ago, the only 2 survivors of ...",English,https://image.tmdb.org/t/p/w500/rzRb63TldOKdKy...,6,https://www.youtube.com/watch?v=2kgJuEX4yhc


In [4]:
# Lọc các dữ liệu trùng lặp
df = df.drop_duplicates(subset=['title'])
len(df)

9652

In [6]:
# Lưu dữ liệu vào file csv
df.to_csv('data_done_NEW.csv', index=False)


In [7]:
# Lọc các title có len trên 100 ký tự
df = df[df['title'].str.len() < 100]
len(df)

8654

In [8]:
df.to_csv('data_done_NEW_1.csv', index=False)

`Chuyển đổi kiểu dữ liệu của cột 'year' sang kiểu số nguyên (integer)`

In [7]:
df = df.dropna(subset=['year'])
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['year'])

df['year'] = df['year'].astype(int) 

`Kiểm tra df có giá trị trùng không xóa các dữ liệu trùng đó`


In [4]:
df.duplicated(subset=['title', 'year']).sum()
df = df.drop_duplicates(subset=['title', 'year'], keep='first')
len(df)

8950

In [10]:
df = pd.read_csv('film1.csv', encoding='utf-8')
df

,title,year,genre,plot,language,poster,rating,trailer
0,Shazam! Fury of the Gods,2023,"Action, Comedy, Fantasy","Billy Batson and his foster siblings, who tran...",en,https://image.tmdb.org/t/p/w500//3GrRgt6CiLIUX...,7,https://www.youtube.com/watch?v=gardMnJszqc
1,Avatar: The Way of Water,2022,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,en,https://image.tmdb.org/t/p/w500//t6HIqrRAclMCA...,8,https://www.youtube.com/watch?v=Sw9hdSjzZjc
2,Creed III,2023,"Drama, Action","After dominating the boxing world, Adonis Cree...",en,https://image.tmdb.org/t/p/w500//cvsXj3I9Q2iyy...,7,https://www.youtube.com/watch?v=qphf_-PPaaI
3,65,2023,"Science Fiction, Adventure, Thriller, Action","65 million years ago, the only 2 survivors of ...",en,https://image.tmdb.org/t/p/w500//rzRb63TldOKdK...,6,https://www.youtube.com/watch?v=2kgJuEX4yhc
4,Murder Mystery 2,2023,"Comedy, Mystery, Action","After starting their own detective agency, Nic...",en,https://image.tmdb.org/t/p/w500//s1VzVhXlqsevi...,7,https://www.youtube.com/watch?v=jeVNn7-FQ64
...,...,...,...,...,...,...,...,...
8950,Chrome Angels,2009,"Action, Thriller, Science Fiction",Welcome to Elliotsville. It’s like your typica...,en,https://image.tmdb.org/t/p/w500//dyGNetZHUcxAP...,4,https://www.youtube.com/watch?v=VTIteJVVT_g
8951,Death Kappa,2010,"Action, Science Fiction, Comedy, Fantasy, Horror",When military experiments go haywire and trigg...,ja,https://image.tmdb.org/t/p/w500//1N0XIm84Dmi9C...,4,https://www.youtube.com/watch?v=_mFVu514iic
8952,Muddy,2021,"Action, Thriller","Muthu and his cousin Karthi have fallen out, b...",ml,https://image.tmdb.org/t/p/w500//jNWrDYMSstert...,6,NaN
8953,Border Town,2009,"Action, Adventure, Thriller",In the five years since human traffickers kidn...,en,https://image.tmdb.org/t/p/w500//lINMjsg4vrMOT...,6,NaN


In [4]:
# Đếm số lượng các ngôn ngữ phim
df['language'].value_counts()

language
en    5274
ja    1018
cn     416
zh     365
hi     333
ko     250
ta     227
fr     185
it     154
te     126
es     107
ru      81
de      63
ml      48
th      47
sv      24
pl      23
id      22
no      19
tr      19
nl      18
da      18
tl      16
pt      16
kn      12
ar      12
ms      10
bn       9
fi       8
vi       5
fa       3
uk       3
ur       2
is       2
pa       2
km       2
ro       2
et       1
af       1
hy       1
hu       1
sq       1
he       1
bo       1
el       1
cs       1
mn       1
bs       1
mi       1
lv       1
xx       1
Name: count, dtype: int64

In [11]:
# Thay thế các ngôn ngữ viết tắt thành ngôn ngữ đầy đủ
df['language'] = df['language'].replace(['en', 'vi', 'ja', 'ko', 'zh', 'th', 'fr', 'de', 'ru', 'es', 'it', 'hi', 'ar', 'tr', 'pt', 'cn', 'id', 'ta', 'fa', 'ml', 'te', 'bn', 'ur', 'pa', 'mr', 'sv', 'nl', 'hu', 'pl', 'da', 'fi', 'no', 'ro', 'cs', 'he', 'uk', 'el', 'ca', 'sr', 'bg', 'sk', 'sl', 'hr', 'et', 'lv', 'lt', 'is', 'af', 'sq', 'sw', 'hy', 'ka', 'mk', 'ms', 'la', 'eu', 'gl', 'be', 'az', 'bs', 'eo', 'ga', 'gu', 'ht', 'jv', 'kn', 'ku', 'lb', 'mt', 'ne', 'si', 'so', 'sq', 'tl', 'ur', 'uz', 'vi', 'yi', 'yo', 'zu'], ['English', 'Vietnamese', 'Japanese', 'Korean', 'Chinese', 'Thai', 'French', 'German', 'Russian', 'Spanish', 'Italian', 'Hindi', 'Arabic', 'Turkish', 'Portuguese', 'Chinese', 'Indonesian', 'Tamil', 'Persian', 'Malayalam', 'Telugu', 'Bengali', 'Urdu', 'Punjabi', 'Marathi', 'Swedish', 'Dutch', 'Hungarian', 'Polish', 'Danish', 'Finnish', 'Norwegian', 'Romanian', 'Czech', 'Hebrew', 'Ukrainian', 'Greek', 'Catalan', 'Serbian', 'Bulgarian', 'Slovak', 'Slovenian', 'Croatian', 'Estonian', 'Latvian', 'Lithuanian', 'Icelandic', 'Afrikaans', 'Albanian', 'Swahili', 'Armenian', 'Georgian', 'Macedonian', 'Malay', 'Latin', 'Basque', 'Galician', 'Belarusian', 'Azerbai jani', 'Bosnian', 'Esperanto', 'Irish', 'Gujarati', 'Haitian', 'Javanese', 'Kannada', 'Kurdish', 'Luxembourgish', 'Maltese', 'Nepali', 'Sinhala', 'Somali', 'Albanian', 'Tagalog', 'Urdu', 'Uzbek', 'Vietnamese', 'Yiddish', 'Yoruba', 'Zulu']) 


In [12]:
# lưu dữ liệu vào file csv
df.to_csv('film2.csv', index=False)
